In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from pytesseract import image_to_data
import imutils
import os
import glob
from collections import defaultdict

In [2]:
def show_img(img, title='', cmap='gray'):
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.show()

In [3]:
TABLE_SIZE = 615

In [4]:
template_img = cv.imread('./board_template.jpg', 0)
template_img = cv.resize(template_img, None, fx=0.3, fy=0.3)

# show_img(template_img, 'template')

In [5]:
# https://docs.opencv.org/3.4/da/d6e/tutorial_py_geometric_transformations.html


def crop_board(template_img):
    pts1 = np.float32([[295, 110], [921, 125], [280, 801], [918, 803]])
    pts2 = np.float32([[0, 0], [TABLE_SIZE, 0], [0, TABLE_SIZE], [TABLE_SIZE, TABLE_SIZE]])

    M = cv.getPerspectiveTransform(pts1, pts2)
    return cv.warpPerspective(template_img, M, (TABLE_SIZE, TABLE_SIZE))

In [6]:
# https://docs.opencv.org/3.4/d1/de0/tutorial_py_feature_homography.html


def match_template(test_img):
    MIN_MATCH_COUNT = 10

    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(template_img, None)
    kp2, des2 = sift.detectAndCompute(test_img, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good.append(m)

    if len(good) <= MIN_MATCH_COUNT:
        print(f'Not enough matches are found - {len(good)}/{MIN_MATCH_COUNT}')
        exit(1)

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    M, _ = cv.findHomography(dst_pts, src_pts, cv.RANSAC, 5.0)

    return cv.warpPerspective(test_img, M, (template_img.shape[1], template_img.shape[0]))

In [7]:
def get_test_img(img_path):
    test_img = cv.imread(img_path, 0)
    test_img = cv.resize(test_img, None, fx=0.3, fy=0.3)
    # show_img(test_img, 'test')

    test_img = match_template(test_img)
    # show_img(test_img, 'warped')

    test_img = crop_board(test_img)
    # show_img(test_img, 'cropped')

    return test_img

In [8]:
class Line:
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2


class Point:
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))

    def get_point_as_tuple(self):
        return (self.x, self.y)


class Patch:
    def __init__(self, image_patch, line_idx, column_idx):
        self.image_patch = image_patch
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.letter = None

    def set_letter(self, letter):
        self.letter = letter

In [9]:
def get_patches(lines, columns, image, show_patches=False):

    def crop_patch(image_, x_min, y_min, x_max, y_max):
        return image_[y_min: y_max, x_min: x_max].copy()

    def draw_patch(image_, x_min, y_min, x_max, y_max, color: tuple = (255, 0, 255)):
        cv.rectangle(image_, (x_min, y_min), (x_max, y_max),
                     color=color, thickness=2)

    assert image.ndim == 2

    if show_patches:
        image_color = np.dstack((image, image, image))

    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)

    patches = []
    step = 5
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            current_line = lines[line_idx]
            next_line = lines[line_idx + 1]

            y_min = current_line.point_1.y + step
            y_max = next_line.point_1.y - step

            current_col = columns[col_idx]
            next_col = columns[col_idx + 1]
            x_min = current_col.point_1.x + step
            x_max = next_col.point_1.x - step

            patch = Patch(image_patch=crop_patch(image, x_min, y_min, x_max, y_max),
                          line_idx=line_idx, column_idx=col_idx)

            if show_patches:
                draw_patch(image_color, x_min, y_min, x_max, y_max)

            patches.append(patch)

    if show_patches:
        show_img(image_color)

    return patches

In [10]:
def get_lines():
    step = TABLE_SIZE / 15

    vertical_lines = []
    horizontal_lines = []

    for i in range(0, 16):
        x_min = np.uint64(i*step)
        line = Line(Point(x=x_min, y=0), Point(x=x_min, y=5000))
        vertical_lines.append(line)
    for i in range(0, 16):
        y_min = np.uint64(i*step)
        line = Line(Point(x=0, y=y_min), Point(x=5000, y=y_min))
        horizontal_lines.append(line)

    return vertical_lines, horizontal_lines

In [11]:
# https://pyimagesearch.com/2021/11/22/improving-ocr-results-with-basic-image-processing/


def preprocess_patch(img):
    img = cv.resize(img, None, fx=2, fy=2, interpolation=cv.INTER_CUBIC)
    # show_img(img, title='Resize')

    blur = cv.GaussianBlur(img, (5, 5), 0)
    # show_img(blur, title='Blur')

    _, thresh = cv.threshold(blur, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
    # show_img(thresh, title='Otsu')

    cnts = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    chars = []
    for c in cnts:
        (x, y, w, h) = cv.boundingRect(c)
        if (w >= 5) and (h >= 30) and (0 < x <= 35) and (0 < y <= 30):
            chars.append(c)

    if len(chars) == 0:
        return None

    chars = np.vstack([chars[i] for i in range(0, len(chars))])
    hull = cv.convexHull(chars)

    mask = np.zeros(img.shape, dtype='uint8')
    cv.drawContours(mask, [hull], -1, 255, -1)
    mask = cv.dilate(mask, None, iterations=1)
    # show_img(mask, title='Mask')

    final = cv.bitwise_and(thresh, thresh, mask=mask)
    final = cv.bitwise_not(final)
    # show_img(final, title='Final')

    return final

In [12]:
def get_letter(patch):
    data = image_to_data(patch, output_type='data.frame', config='--psm 10')
    pred = data.text.iloc[-1]

    # high chance to be I
    if pred is None or pred != pred:  # nan
        return 'I'

    if pred == '|' or pred == ']' or pred == '[':
        return 'I'

    if pred == 0:
        return 'O'

    if pred == 'x':
        return 'X'

    if type(pred) == str:
        if len(pred) > 1:
            # find first uppercase letter
            for letter in pred:
                if letter.isupper():
                    return letter
            return None

        if not pred.isupper():
            return None

        return pred

    return None

In [13]:
def get_coordinates(i, j):
    return str(i+1), chr(ord('A')+j)

In [14]:
def mean_ssd(img_1, img_2):
    img_1 = np.float32(img_1)
    img_2 = np.float32(img_2)
    return np.mean((img_1 - img_2) ** 2)

In [15]:
blank = cv.imread('./blank.jpg', 0)


def is_blank(patch):
    if mean_ssd(patch, blank) < 300:
        return True
    return False

In [16]:
points = {
    'A': 1, 'B': 3, 'C': 3, 'D': 2, 'E': 1, 'F': 4, 'G': 2,
    'H': 4, 'I': 1, 'J': 8, 'K': 5, 'L': 1, 'M': 3, 'N': 1,
    'O': 1, 'P': 3, 'Q': 10, 'R': 1, 'S': 1, 'T': 1, 'U': 1,
    'V': 4, 'W': 4, 'X': 8, 'Y': 4, 'Z': 10, '?': 0
}

In [17]:
triple_word = [(0, 0), (0, 7), (0, 14),
               (7, 0), (7, 14),
               (14, 0), (14, 7), (14, 14)]

double_word = [(1, 1), (2, 2), (3, 3), (4, 4), (10, 10), (11, 11), (12, 12), (13, 13),
               (1, 13), (2, 12), (3, 11), (4, 10), (10, 4), (11, 3), (12, 2), (13, 1), (7, 7)]

triple_letter = [(1, 5), (1, 9), (5, 1), (5, 5), (5, 9), (5, 13),
                 (9, 1), (9, 5), (9, 9), (9, 13), (13, 5), (13, 9)]

double_letter = [(0, 3), (0, 11), (2, 6), (2, 8), (3, 0), (3, 7), (3, 14),
                 (6, 2), (6, 6), (6, 8), (6, 12), (7, 3), (7, 11), (8, 2), (8, 6), (8, 8), (8, 12),
                 (11, 0), (11, 7), (11, 14), (12, 6), (12, 8), (14, 3), (14, 11)]


letter_mult = [[1 for _ in range(15)] for _ in range(15)]
for i, j in triple_letter:
    letter_mult[i][j] = 3
for i, j in double_letter:
    letter_mult[i][j] = 2

word_mult = [[1 for _ in range(15)] for _ in range(15)]
for i, j in triple_word:
    word_mult[i][j] = 3
for i, j in double_word:
    word_mult[i][j] = 2

In [18]:
def valid_neighbour(i, j, board, visited):
    if 0 <= i < 15 and 0 <= j < 15 and board[i][j] is not None and not visited[(i, j)]:
        return True
    return False


def get_score(board, tiles):
    visited = defaultdict(lambda: False)  # visited tiles
    score = 0

    # main word created
    if len(tiles) > 1:
        word_dir_i, word_dir_j = 1, 1
        for i in range(0, len(tiles)-1):
            word_dir_i = min(word_dir_i, tiles[i+1][0] - tiles[i][0])
            word_dir_j = min(word_dir_j, tiles[i+1][1] - tiles[i][1])

        tile_idx = 0
        mult = 1
        first_i, first_j = tiles[0]
        i, j = first_i, first_j
        while valid_neighbour(i, j, board, visited):
            letter = board[i][j]
            if tile_idx < len(tiles) and tiles[tile_idx][0] == i and tiles[tile_idx][1] == j:
                score += points[letter] * letter_mult[i][j]
                mult *= word_mult[i][j]
                tile_idx += 1
            else:
                score += points[letter]
            visited[(i, j)] = True
            i += word_dir_i
            j += word_dir_j

        # check the other direction
        word_dir_i = -word_dir_i
        word_dir_j = -word_dir_j
        i, j = first_i+word_dir_i, first_j+word_dir_j
        while valid_neighbour(i, j, board, visited):
            letter = board[i][j]
            score += points[letter]
            visited[(i, j)] = True
            i += word_dir_i
            j += word_dir_j

        score *= mult

    dir_i = [0, 1, 0, -1]
    dir_j = [-1, 0, 1, 0]

    # additional created words
    for (i, j) in tiles:
        for d in range(4):
            partial_score = 0
            mult = 1
            new_i, new_j = i+dir_i[d], j+dir_j[d]
            if valid_neighbour(new_i, new_j, board, visited):
                letter = board[i][j]
                partial_score += points[letter] * letter_mult[i][j]
                mult *= word_mult[i][j]
            while valid_neighbour(new_i, new_j, board, visited):
                letter = board[new_i][new_j]
                partial_score += points[letter]
                new_i += dir_i[d]
                new_j += dir_j[d]
                visited[(i, j)] = True
            score += (partial_score * mult)

    if len(tiles) >= 7:
        score += 50

    return score

In [24]:
# prepare output directory
output_dir = './output'
try:
    for file_name in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, file_name))
except: 
    os.mkdir(output_dir)

In [27]:
def score_games(input_folder):
    for game in range(1, 6):
        board = [[None for _ in range(15)] for _ in range(15)]

        for img_path in glob.glob(f'{input_folder}/{game}_*.jpg'):
            output_file, _ = os.path.splitext(os.path.basename(img_path))
            f = open(f'./output/{output_file}.txt', 'a+')

            test_img = get_test_img(img_path)
            vertical_lines, horizontal_lines = get_lines()
            patches = get_patches(horizontal_lines, vertical_lines, test_img)

            # new tiles added this turn
            tiles = []

            for patch in patches:
                i, j = patch.line_idx, patch.column_idx

                # letter from previous turn
                if board[i][j] is not None:
                    continue

                p = patch.image_patch

                mean, stddev = cv.meanStdDev(p)
                norm_mean = stddev / mean * 100

                # empty cell or blank tile
                if norm_mean[0][0] < 10:
                    if is_blank(p):
                        letter = '?'
                        board[i][j] = letter
                        coords = ''.join(get_coordinates(i, j))
                        f.write(f'{coords} {letter}\n')
                        patch.set_letter(letter)
                        tiles.append((i, j))
                    continue

                p = preprocess_patch(p)

                # blue/red cell
                if p is None:
                    continue

                letter = get_letter(p)
                if letter is not None:
                    board[i][j] = letter
                    coords = ''.join(get_coordinates(i, j))
                    f.write(f'{coords} {letter}\n')
                    patch.set_letter(letter)
                    tiles.append((i, j))

            score = get_score(board, tiles)
            f.write(str(score))

            f.close()

In [28]:
score_games('./train_data')